## Notebook with data reading and processing steps

In [1]:
import rasterio as rs 
# import xarray as xr
from rasterio.windows import Window
from rasterio.transform import xy
from pandas import DataFrame
import numpy as np
from numpy import array,meshgrid,arange,log

In [2]:
src_file="/Users/cenv1069/Documents/data/datasets/JRC/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R8_C19/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R8_C19.tif"

In [ ]:
# using rasterio : 

chunk_width = chunk_height = 1000

n = 0

full_height = int()
full_width = int()

window_=Window(n, n, (n+1)*chunk_height, (n+1)*chunk_width)

out = DataFrame()
with rs.open(src_file) as src : 
    band1 = src.read(1
                     ,window=window_
                     )
    
    full_height = band1.shape[0]
    full_width = band1.shape[1]
    
    print(full_height/chunk_height)
    print(full_width/chunk_width)
    print(src.nodatavals)

# width_floor = np.floor(full_width/chunk_width)
# height_floor = np.floor(full_height/chunk_height)

# with rs.open(src_file) as src : 
#     band1 = src.read(1
#                      ,window=window_
#                      )
#     print(src.transform)

#     cols, rows = meshgrid(arange(width), arange(height))
#     xs, ys = xy(transform = src.transform, rows=rows, cols=cols)

#     lons = array(xs)
#     lats = array(ys)

#     out = DataFrame({"band_var" : array(band1).flatten()
#                             ,'lon': lons.flatten()
#                             ,'lat': lats.flatten()})
    
    # out.drop(index=out.loc[out.band_var==src.nodatavals].index,inplace=True)
    # out.drop(index=out.loc[out.band_var<=0].index,inplace=True)



In [ ]:
out

## Dask 


In [3]:
import dask as dk
import dask.array as da
import dask.dataframe as dd
# import dask_image as dki
import xarray as xr
import rioxarray as rx

In [50]:
# big_file = dk.imread.imread(src_file)

In [15]:
big_file = rx.open_rasterio(src_file,chunks=(1,1000,1000),lock=False,variable="band")

In [16]:
big_file

<xarray.DataArray (band: 1, y: 100000, x: 100000)> Size: 10GB
dask.array<open_rasterio-52a411cebe5cb97342cccb6fa5e7bb06<this-array>, shape=(1, 100000, 100000), dtype=uint8, chunksize=(1, 1000, 1000), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 800kB -4.1e+04 -4.098e+04 ... 9.59e+05 9.59e+05
  * y            (y) float64 800kB 2e+06 2e+06 2e+06 2e+06 ... 1e+06 1e+06 1e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     255
    scale_factor:   1.0
    add_offset:     0.0

In [17]:
nodataval = big_file.attrs["_FillValue"]

big_file[big_file.band!=nodataval]

<xarray.DataArray (band: 1, y: 100000, x: 100000)> Size: 10GB
dask.array<getitem, shape=(1, 100000, 100000), dtype=uint8, chunksize=(1, 1000, 1000), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 800kB -4.1e+04 -4.098e+04 ... 9.59e+05 9.59e+05
  * y            (y) float64 800kB 2e+06 2e+06 2e+06 2e+06 ... 1e+06 1e+06 1e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     255
    scale_factor:   1.0
    add_offset:     0.0

In [18]:
big_file = big_file.drop_indexes(coord_names=["x","y","band"])

In [25]:
# big_file[big_file.band!=nodataval]
big_file

<xarray.DataArray (band: 1, y: 100000, x: 100000)> Size: 10GB
dask.array<open_rasterio-52a411cebe5cb97342cccb6fa5e7bb06<this-array>, shape=(1, 100000, 100000), dtype=uint8, chunksize=(1, 1000, 1000), chunktype=numpy.ndarray>
Coordinates:
    band         (band) int64 8B 1
    x            (x) float64 800kB -4.1e+04 -4.098e+04 ... 9.59e+05 9.59e+05
    y            (y) float64 800kB 2e+06 2e+06 2e+06 2e+06 ... 1e+06 1e+06 1e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     255
    scale_factor:   1.0
    add_offset:     0.0

In [19]:
big_file.dims

('band', 'y', 'x')

In [20]:
big_file.where(big_file.band!=255,drop=True)

<xarray.DataArray (band: 1, y: 100000, x: 100000)> Size: 40GB
dask.array<where, shape=(1, 100000, 100000), dtype=float32, chunksize=(1, 1000, 1000), chunktype=numpy.ndarray>
Coordinates:
    band         (band) int64 8B 1
    x            (x) float64 800kB -4.1e+04 -4.098e+04 ... 9.59e+05 9.59e+05
    y            (y) float64 800kB 2e+06 2e+06 2e+06 2e+06 ... 1e+06 1e+06 1e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     255
    scale_factor:   1.0
    add_offset:     0.0

In [22]:
# big_file_dk = da.from_array(big_file,chunks=(1,1000,1000))

In [24]:
dk.array.reshape(big_file,shape=(-1,3))

IndexError: Bad index.  Must be integer-like: 3333333333.3333335